In [13]:
from torchsummary import summary
from datetime import datetime
import torchvision.transforms as transforms
import torch.nn as nn
import torch
import random
import numpy as np

from src.nn.regression_dataset import RegressionDataset
from src.nn.to_tensor import ToTensor
from src.nn.create_data_loaders import create_data_loaders
from src.nn.cnn_regressor import CNNRegressor
from src.nn.weighted_mse_loss import WeightedMSELoss
from src.nn.training import training
from src.nn.plot_losses import plot_losses
from src.data.synthMRWregul import synthMRWregul
import src.ctes.num_ctes as nctes
import src.ctes.str_ctes as sctes

In [14]:
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [15]:
data_path = "../../data/MRW.npz"
transform = ToTensor()
sample_size = nctes.LEN_SAMPLE

data = RegressionDataset(data_path, transform, sample_size)

In [16]:
batch_size = 6
valid_size = 0.2
test_size = 0.2

train_loader, valid_loader, test_loader = create_data_loaders(batch_size, valid_size, test_size, data)

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device ' + str(device))

Using device cuda


In [18]:
model = CNNRegressor(input_size=sample_size)
model.to(device=device)

summary(model, (1, sample_size))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 32768]           --
|    └─Conv1d: 2-1                       [-1, 16, 32768]           16
|    └─BatchNorm1d: 2-2                  [-1, 16, 32768]           32
|    └─ReLU: 2-3                         [-1, 16, 32768]           --
├─Sequential: 1-2                        [-1, 32, 32767]           --
|    └─Conv1d: 2-4                       [-1, 32, 32767]           1,024
|    └─BatchNorm1d: 2-5                  [-1, 32, 32767]           64
|    └─ReLU: 2-6                         [-1, 32, 32767]           --
├─AvgPool1d: 1-3                         [-1, 32, 16384]           --
├─Sequential: 1-4                        [-1, 64, 16381]           --
|    └─Conv1d: 2-7                       [-1, 64, 16381]           8,192
|    └─BatchNorm1d: 2-8                  [-1, 64, 16381]           128
|    └─ReLU: 2-9                         [-1, 64, 16381]           --
├─AvgPoo

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 32768]           --
|    └─Conv1d: 2-1                       [-1, 16, 32768]           16
|    └─BatchNorm1d: 2-2                  [-1, 16, 32768]           32
|    └─ReLU: 2-3                         [-1, 16, 32768]           --
├─Sequential: 1-2                        [-1, 32, 32767]           --
|    └─Conv1d: 2-4                       [-1, 32, 32767]           1,024
|    └─BatchNorm1d: 2-5                  [-1, 32, 32767]           64
|    └─ReLU: 2-6                         [-1, 32, 32767]           --
├─AvgPool1d: 1-3                         [-1, 32, 16384]           --
├─Sequential: 1-4                        [-1, 64, 16381]           --
|    └─Conv1d: 2-7                       [-1, 64, 16381]           8,192
|    └─BatchNorm1d: 2-8                  [-1, 64, 16381]           128
|    └─ReLU: 2-9                         [-1, 64, 16381]           --
├─AvgPoo

In [19]:
c1_weight      = 1 / (0.8)**2
c2_weight      = 1 / (0.08)**2
L_weight       = 1 / (5000)**2
epsilon_weight = 1 / (4.5)**2
mse_weights    = torch.FloatTensor([c1_weight, c2_weight, L_weight, epsilon_weight]).to(device=device)
criterion = WeightedMSELoss(mse_weights)
print(criterion)

WeightedMSELoss(weights=tensor([1.5625e+00, 1.5625e+02, 4.0000e-08, 4.9383e-02], device='cuda:0'))


In [20]:
params = model.parameters()
lr = 0.001

optimizer = torch.optim.Adam(params, lr)
print(optimizer)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


In [21]:
n_epochs = 20

In [22]:
timestamp = datetime.now().strftime('%Y_%m_%d__%H_%M_%S')
hyperparams_path = f"../../data/hyperparams_{timestamp}.npz"
model_path = f"../../data/model_{timestamp}.pt"
losses_path = f"../../data/losses_{timestamp}.npz"

In [23]:
np.savez(hyperparams_path, 
         len=len(data), 
         test_size=test_size, 
         valid_size=valid_size, 
         epochs=n_epochs, 
         batch_size=batch_size, 
         criterion=str(criterion), 
         optimizer=str(optimizer), 
         lr=lr,
         seed=seed)

In [24]:
train_losses, valid_losses = training(n_epochs, train_loader, valid_loader, model, criterion, optimizer, device, model_path)

Epoch 1/20 ...
Training ...


100%|███████████████████████████████████████| 1366/1366 [02:33<00:00,  8.92it/s]


Validating ...


100%|█████████████████████████████████████████| 342/342 [00:13<00:00, 25.22it/s]


Epoch: 1 	training Loss: 465.591243 	validation Loss: 175.311626
Validation loss decreased (inf --> 175.311626).  Saving model ...
Epoch 2/20 ...
Training ...


 12%|████▋                                   | 162/1366 [00:18<02:19,  8.65it/s]


KeyboardInterrupt: 

In [ ]:
# train_losses = [2403.609713, 232.189727, 99.660317, 86.992496, 83.993411, 80.575210, 78.289100, 76.117352, 71.029479, 67.108203, 62.995093, 60.440386, 57.365357, 54.026526, 52.620844, 48.840387, 47.632184]
# valid_losses = [434.089203, 134.722389, 98.011503, 88.664742, 86.704399, 84.115528, 79.012016, 76.117352, 72.722704, 69.212138, 67.791914, 63.573414, 58.499450, 58.289474, 59.753458, 62.972973, 52.344090]

In [ ]:
np.savez(losses_path , train=np.array(train_losses), val=np.array(valid_losses))

In [ ]:
plot_losses(len(train_losses), [train_losses, valid_losses], ["Train", "Val"])